### What to do
Scrape the websites in order to extract the following information:
- Name 
- Logo
- Location 
- Themes (i.e. children, homeless, medicine...)
- Description / mission
- URL of their website
- Year of foundation (This information is sometimes implicit: can be computed based on Number of year of activity)

### What to use
- Requests
- BeautifulSoup
- Scrapy

### Websites to scrape
urls = [
    "https://www.charitynavigator.org",
    "https://www.globalgiving.org",
    "https://www.guidestar.org",
]

### Charity Navigator API
https://charity-navigator.stellate.io

In [14]:
## Fetch data from Charity Navigator API

import os
from dotenv import load_dotenv
import requests
import pandas as pd
import json

# Load environment variables from .env file
load_dotenv()

# Access the variables
charity_navigator_key = os.getenv('CHARITY_NAVIGATOR')

CHARITY_NAVIGATOR_ENDPOINT = 'https://data.charitynavigator.org/'

# Fetch data from Charity Navigator
def fetch_charity_data(count):
    QUERY = """
    query {
        publicSearchFaceted(term: "", from: %d) {
            size
            from
            term
            result_count
            results {
                ein
                name
                mission
                organization_url
                charity_navigator_url
                encompass_score
                encompass_star_rating
                encompass_publication_date
                cause
                street
                street2
                city
                state
                zip
                country
                highest_level_advisory
                encompass_rating_id
            }
        }
    }
    """ % count

    headers = {
        "Stellate-Api-Token": charity_navigator_key,
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.post(
        CHARITY_NAVIGATOR_ENDPOINT,
        headers=headers,
        json={"query": QUERY}
    )
    # Raise an error if the request fails
    response.raise_for_status()
    
    # Return the JSON response
    return response.json()

results = []

for i in range(0, 20, 10):
   data = fetch_charity_data(i)
   print(data)
   results.append(data)

# Save the results to a JSON file
with open('charity_navigator_data.json', 'w') as f:
    f.write(json.dumps(results, indent=2))

{'data': {'publicSearchFaceted': {'size': 10, 'from': 0, 'term': '', 'result_count': 10000, 'results': [{'ein': '262414132', 'name': 'Mercy Ships International ', 'mission': 'We follow the 2000-year-old model of Jesus, bringing hope and healing to the forgotten poor. \n\nMercy Ships is a global charity that has operated hospital ships in developing nations since its founding in 1978. Mercy Ships brings hope and healing to the forgotten poor by mobilizing people and resources worldwide, and serving all people without regard for race, gender, or religion. Mercy Ships programs promote health and well-being by serving the urgent surgical needs of the forgotten poor and empowering developing communities. Since its founding, Mercy Ships has performed more than delivered services valued at over $1.43 billion and impacting more than 16.3 million people as direct beneficiaries.', 'organization_url': 'WWW.MERCYSHIPS.ORG', 'charity_navigator_url': 'https://charitynavigator.org/ein/262414132', 'en

In [ ]:
import pandas as pd
import json

## Filter data from charity_navigator_data.csv file to get the required data

# charity_navigator_data.csv
charity_navigator_data = pd.read_csv("charity_navigator_data.csv")

print(type(charity_navigator_data))
print(len(charity_navigator_data))

data = charity_navigator_data["data"]
print(data)

# Filter data
charity_navigator_data_filtered = data[
    ["name", "mission", "organization_url", "cause", "state", "country"]
]
# Read the JSON file
with open("charity_navigator_data.json", "r") as f:
    charity_data_json = json.load(f)

# Convert JSON data to DataFrame
charity_data_list = [
    item["data"]["publicSearchFaceted"]["results"] for item in charity_data_json
]
charity_data_flat = [item for sublist in charity_data_list for item in sublist]
charity_navigator_data_filtered = pd.DataFrame(charity_data_flat)

print(charity_navigator_data_filtered.head())

<class 'pandas.core.frame.DataFrame'>
1001
0       {'publicSearchFaceted': {'size': 10, 'from': 0...
1       {'publicSearchFaceted': {'size': 10, 'from': 1...
2       {'publicSearchFaceted': {'size': 10, 'from': 2...
3       {'publicSearchFaceted': {'size': 10, 'from': 3...
4       {'publicSearchFaceted': {'size': 10, 'from': 4...
                              ...                        
996     {'publicSearchFaceted': {'size': 10, 'from': 9...
997     {'publicSearchFaceted': {'size': 10, 'from': 9...
998     {'publicSearchFaceted': {'size': 10, 'from': 9...
999     {'publicSearchFaceted': {'size': 10, 'from': 9...
1000                                                  NaN
Name: data, Length: 1001, dtype: object


KeyError: "None of [Index(['name', 'mission', 'organization_url', 'cause', 'state', 'country'], dtype='object')] are in the [index]"